### Test case LVV-T150: Verify implementation of Maintain Archive Publicly Accessible

Verify that prior data releases remain accessible.

We will demonstrate this by loading catalogs from the TAP service in the public RSP instance at data.lsst.cloud.

In [1]:
from lsst.rsp import get_tap_service, retrieve_query

In [2]:
service = get_tap_service("tap")

#### Retrieve an `Object` catalog from DP0.2:

Execute a cone search with a small radius from the DP0.2 `Object` catalog.

In [3]:
center_ra = 62
center_dec = -37
radius = 0.01

str_center_coords = str(center_ra) + ", " + str(center_dec)
str_radius = str(radius)

query = "SELECT coord_ra, coord_dec "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', " + str_center_coords + ", " + str_radius + ")) = 1 "\
        "AND detect_isPrimary = 1"
print(query)

SELECT coord_ra, coord_dec FROM dp02_dc2_catalogs.Object WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), CIRCLE('ICRS', 62, -37, 0.01)) = 1 AND detect_isPrimary = 1


In [4]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Job phase is COMPLETED


In [5]:
results = job.fetch_result().to_table().to_pandas()
print(len(results))

173


In [6]:
results

,coord_ra,coord_dec
0,61.989258,-37.005119
1,61.997438,-37.005796
2,62.003285,-37.006248
3,61.998238,-37.007123
4,62.003341,-37.002536
...,...,...
168,61.990251,-36.996347
169,61.989260,-36.995563
170,61.990184,-36.995566
171,61.992685,-36.998426


We see that the DP0.2 data are available and well-formed.

### Retrieve an `Object` catalog from DP1

Now execute a query of DP1 data using the same TAP service:

In [7]:
query = """SELECT coord_ra, coord_dec
        FROM dp1.Object
        WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),
        CIRCLE('ICRS', 53, -28, 0.01)) = 1"""
print(query)

SELECT coord_ra, coord_dec
        FROM dp1.Object
        WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),
        CIRCLE('ICRS', 53, -28, 0.01)) = 1


In [8]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'
results = job.fetch_result().to_table()

Job phase is COMPLETED


In [9]:
results

coord_ra,coord_dec
deg,deg
float64,float64
52.999950842186635,-28.009791369795494
53.000647668093336,-28.00041790185584
52.99865025242912,-28.007343580438622
53.00440835952543,-28.00706527664946
52.99728049236805,-28.004763783217346
52.998611316150786,-28.007966170045258
52.99610446568829,-28.00392847688764
52.994574524381925,-27.997372314676916


## Results

We have demonstrated that catalogs from multiple data releases are available to users in the Science Platform. Thus, we deem the result of this test a **Pass**.